# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 18 2022 9:17AM,244447,10,6858805,"Senseonics, Incorporated",Released
1,Aug 18 2022 9:11AM,244446,10,0085941493,ISDIN Corporation,Released
2,Aug 18 2022 9:11AM,244446,10,0085941492,ISDIN Corporation,Released
3,Aug 18 2022 9:11AM,244446,10,0085941494,ISDIN Corporation,Released
4,Aug 18 2022 9:11AM,244446,10,0085941496,ISDIN Corporation,Released
5,Aug 18 2022 9:11AM,244446,10,0085941491,ISDIN Corporation,Released
6,Aug 18 2022 9:11AM,244446,10,0085941499,ISDIN Corporation,Released
7,Aug 18 2022 9:11AM,244446,10,0085941498,ISDIN Corporation,Released
8,Aug 18 2022 9:11AM,244446,10,0085941500,ISDIN Corporation,Released
9,Aug 18 2022 9:11AM,244446,10,0085941501,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
45,244440,Released,12
46,244443,Released,3
47,244445,Released,8
48,244446,Released,29
49,244447,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
244440,NaN,NaN,12.0
244443,NaN,NaN,3.0
244445,NaN,NaN,8.0
244446,NaN,NaN,29.0
244447,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244330,4.0,16.0,0.0
244351,0.0,0.0,20.0
244354,3.0,0.0,1.0
244355,23.0,0.0,2.0
244357,9.0,1.0,3.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244330,4,16,0
244351,0,0,20
244354,3,0,1
244355,23,0,2
244357,9,1,3


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244330,4,16,0
1,244351,0,0,20
2,244354,3,0,1
3,244355,23,0,2
4,244357,9,1,3


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244330,4,16,
1,244351,,,20
2,244354,3,,1
3,244355,23,,2
4,244357,9,1,3


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 18 2022 9:17AM,244447,10,"Senseonics, Incorporated"
1,Aug 18 2022 9:11AM,244446,10,ISDIN Corporation
30,Aug 18 2022 8:59AM,244445,10,Bio-PRF
38,Aug 18 2022 8:48AM,244443,10,"Senseonics, Incorporated"
41,Aug 18 2022 8:39AM,244440,10,ISDIN Corporation
53,Aug 18 2022 8:39AM,244439,10,ISDIN Corporation
79,Aug 18 2022 8:39AM,244438,10,ISDIN Corporation
129,Aug 18 2022 8:30AM,244437,20,"Exact-Rx, Inc."
133,Aug 18 2022 8:30AM,244413,10,ISDIN Corporation
138,Aug 18 2022 8:30AM,244425,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 18 2022 9:17AM,244447,10,"Senseonics, Incorporated",,,1
1,Aug 18 2022 9:11AM,244446,10,ISDIN Corporation,,,29
2,Aug 18 2022 8:59AM,244445,10,Bio-PRF,,,8
3,Aug 18 2022 8:48AM,244443,10,"Senseonics, Incorporated",,,3
4,Aug 18 2022 8:39AM,244440,10,ISDIN Corporation,,,12
5,Aug 18 2022 8:39AM,244439,10,ISDIN Corporation,,,26
6,Aug 18 2022 8:39AM,244438,10,ISDIN Corporation,,,50
7,Aug 18 2022 8:30AM,244437,20,"Exact-Rx, Inc.",,,4
8,Aug 18 2022 8:30AM,244413,10,ISDIN Corporation,,,5
9,Aug 18 2022 8:30AM,244425,10,Emerginnova,1,3,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 18 2022 9:17AM,244447,10,"Senseonics, Incorporated",1,,
1,Aug 18 2022 9:11AM,244446,10,ISDIN Corporation,29,,
2,Aug 18 2022 8:59AM,244445,10,Bio-PRF,8,,
3,Aug 18 2022 8:48AM,244443,10,"Senseonics, Incorporated",3,,
4,Aug 18 2022 8:39AM,244440,10,ISDIN Corporation,12,,
5,Aug 18 2022 8:39AM,244439,10,ISDIN Corporation,26,,
6,Aug 18 2022 8:39AM,244438,10,ISDIN Corporation,50,,
7,Aug 18 2022 8:30AM,244437,20,"Exact-Rx, Inc.",4,,
8,Aug 18 2022 8:30AM,244413,10,ISDIN Corporation,5,,
9,Aug 18 2022 8:30AM,244425,10,Emerginnova,1,3,1


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 18 2022 9:17AM,244447,10,"Senseonics, Incorporated",1,,
1,Aug 18 2022 9:11AM,244446,10,ISDIN Corporation,29,,
2,Aug 18 2022 8:59AM,244445,10,Bio-PRF,8,,
3,Aug 18 2022 8:48AM,244443,10,"Senseonics, Incorporated",3,,
4,Aug 18 2022 8:39AM,244440,10,ISDIN Corporation,12,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 18 2022 9:17AM,244447,10,"Senseonics, Incorporated",1.0,NaN,NaN
1,Aug 18 2022 9:11AM,244446,10,ISDIN Corporation,29.0,NaN,NaN
2,Aug 18 2022 8:59AM,244445,10,Bio-PRF,8.0,NaN,NaN
3,Aug 18 2022 8:48AM,244443,10,"Senseonics, Incorporated",3.0,NaN,NaN
4,Aug 18 2022 8:39AM,244440,10,ISDIN Corporation,12.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 18 2022 9:17AM,244447,10,"Senseonics, Incorporated",1.0,0.0,0.0
1,Aug 18 2022 9:11AM,244446,10,ISDIN Corporation,29.0,0.0,0.0
2,Aug 18 2022 8:59AM,244445,10,Bio-PRF,8.0,0.0,0.0
3,Aug 18 2022 8:48AM,244443,10,"Senseonics, Incorporated",3.0,0.0,0.0
4,Aug 18 2022 8:39AM,244440,10,ISDIN Corporation,12.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4643955,170.0,4.0,13.0
12,488813,0.0,2.0,0.0
15,244355,2.0,0.0,23.0
16,488836,20.0,0.0,0.0
17,244406,1.0,0.0,0.0
19,733234,28.0,10.0,11.0
20,977514,97.0,16.0,4.0
21,1710925,7.0,0.0,0.0
22,488754,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4643955,170.0,4.0,13.0
1,12,488813,0.0,2.0,0.0
2,15,244355,2.0,0.0,23.0
3,16,488836,20.0,0.0,0.0
4,17,244406,1.0,0.0,0.0
5,19,733234,28.0,10.0,11.0
6,20,977514,97.0,16.0,4.0
7,21,1710925,7.0,0.0,0.0
8,22,488754,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,170.0,4.0,13.0
1,12,0.0,2.0,0.0
2,15,2.0,0.0,23.0
3,16,20.0,0.0,0.0
4,17,1.0,0.0,0.0
5,19,28.0,10.0,11.0
6,20,97.0,16.0,4.0
7,21,7.0,0.0,0.0
8,22,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,170.0
1,12,Released,0.0
2,15,Released,2.0
3,16,Released,20.0
4,17,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,17,19,20,21,22
Status,,,,,,,,,
Completed,13.0,0.0,23.0,0.0,0.0,11.0,4.0,0.0,0.0
Executing,4.0,2.0,0.0,0.0,0.0,10.0,16.0,0.0,1.0
Released,170.0,0.0,2.0,20.0,1.0,28.0,97.0,7.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,17,19,20,21,22
0,Completed,13.0,0.0,23.0,0.0,0.0,11.0,4.0,0.0,0.0
1,Executing,4.0,2.0,0.0,0.0,0.0,10.0,16.0,0.0,1.0
2,Released,170.0,0.0,2.0,20.0,1.0,28.0,97.0,7.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,17,19,20,21,22
0,Completed,13.0,0.0,23.0,0.0,0.0,11.0,4.0,0.0,0.0
1,Executing,4.0,2.0,0.0,0.0,0.0,10.0,16.0,0.0,1.0
2,Released,170.0,0.0,2.0,20.0,1.0,28.0,97.0,7.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()